In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("data2.csv")
data = data[data['age'] < 100]
data = data[data['gender'] > 0]
data.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,score,gender,age
0,1,1,2,1,1,4,2,3,3,1,19,1,41
1,2,1,1,1,1,2,1,2,1,2,14,1,50
2,2,1,1,1,1,4,1,1,1,2,15,1,23
3,2,1,1,1,1,4,1,1,1,4,17,1,42
4,1,1,1,1,1,2,1,1,1,1,11,2,36


In [3]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data)

data_dev = data[0:700].T
Y_dev = data_dev[11]
X_dev = data_dev[0:n-3]

data_train = data[700:m].T
Y_train = data_train[11]
X_train = data_train[0:n-3]



In [4]:
def init_params():
    W1 = np.random.rand(10, 10) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(3, 10) - 0.5
    b2 = np.random.rand(3, 1) - 0.5
    return W1, b1, W2, b2

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = sigmoid(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def der_sigmoid(x):
    return sigmoid(x) * (1- sigmoid(x))

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max()))
    one_hot_Y[np.arange(Y.size), Y-1] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * der_sigmoid(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [5]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y-1) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            print(A2)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [6]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.01, 10000)

Iteration:  0
[[0.38978277 0.38818647 0.38412133 ... 0.3963958  0.35284071 0.33018009]
 [0.34307616 0.37719245 0.33221057 ... 0.2839017  0.34411391 0.33916508]
 [0.26714107 0.23462107 0.2836681  ... 0.3197025  0.30304539 0.33065483]]
0.5917293233082707
Iteration:  10
[[0.41026819 0.40887646 0.40241925 ... 0.42248123 0.37172328 0.3488226 ]
 [0.34080731 0.37131311 0.33065864 ... 0.28176087 0.34287063 0.33829262]
 [0.2489245  0.21981043 0.26692211 ... 0.2957579  0.28540608 0.31288478]]
0.643984962406015
Iteration:  20
[[0.42976478 0.42838991 0.41980815 ... 0.44728063 0.38961496 0.36653009]
 [0.33801024 0.36513245 0.32864063 ... 0.27902253 0.3411913  0.33696134]
 [0.23222498 0.20647764 0.25155122 ... 0.27369684 0.26919374 0.29650857]]
0.6654135338345865
Iteration:  30
[[0.44839402 0.44676779 0.43638525 ... 0.47084814 0.40666449 0.38343223]
 [0.33475616 0.35885183 0.32624747 ... 0.27577068 0.33916702 0.33529059]
 [0.21684982 0.19438038 0.23736728 ... 0.25338118 0.25416849 0.28127718]]
0.679

In [7]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

0.68